# Installation

In [ ]:
update = False

from os import path

%cd /kaggle/working
!git clone https://github.com/comfyanonymous/ComfyUI.git
%cd ComfyUI
if update:
    get_ipython().system('git pull')
!pip install -r requirements.txt

checkpoints =  '/kaggle/working/ComfyUI/models/checkpoints'
link_path = checkpoints + '/temp-models'
temp_models = '/kaggle/temp/temp-models'

!mkdir /kaggle/temp
!mkdir $temp_models

if not path.exists(link_path):
    get_ipython().system(f'ln -s {temp_models} {checkpoints}')

!mamba install openssh

# Install the node manager
update_manager = False
%cd /kaggle/working/ComfyUI/custom_nodes
!git clone https://github.com/Comfy-Org/ComfyUI-Manager.git
%cd ComfyUI-Manager
%cd /kaggle/working/ComfyUI/custom_nodes
!git clone https://github.com/set-soft/ComfyUI-FLOAT_Optimized.git
%cd ComfyUI-FLOAT_Optimized
%cd /kaggle/working/ComfyUI/custom_nodes
!git clone https://github.com/Kosinkadink/ComfyUI-VideoHelperSuite.git
%cd ComfyUI-VideoHelperSuite.git


if update_manager:
    get_ipython().system('git pull')

# Download Checkpoint

In [ ]:
!mkdir -p /kaggle/working/ComfyUI/models/float
!wget -O /kaggle/working/ComfyUI/models/float/FLOAT.safetensors \
"https://huggingface.co/set-soft/float/resolve/main/FLOAT.safetensors?download=true"


# Launch with Pinggy

In [ ]:
!pip install torchdiffeq
!pip install face-alignment
!pip install opencv-python-headless



In [10]:
!sed -i 's/h264_nvenc/libx264/g' /kaggle/working/ComfyUI/custom_nodes/comfyui-videohelpersuite/videohelpersuite/nodes.py


In [ ]:
# Starting the Web UI with pinggy

from multiprocessing import Process
import sys
import time

!touch log.txt
open('log.txt', 'w').close()

def run_app():
    cmd = f"python /kaggle/working/ComfyUI/main.py & ssh -o StrictHostKeyChecking=no -p 80 -R0:localhost:8188 a.pinggy.io > log.txt"
    get_ipython().system(cmd)
    
def print_url():
    print("waiting for output")
    time.sleep(2)
    sys.stdout.flush()
    
    found = False
    with open('log.txt', 'r') as file:
        end_word = '.pinggy.link'
        for line in file:
            start_index = line.find("http:")
            if start_index != -1:
                end_index = line.find(end_word, start_index)
                if end_index != -1:
                    print("😁 😁 😁")
                    print("URL: " + line[start_index:end_index + len(end_word)])
                    print("😁 😁 😁")
                    found = True
    if not found:
        print_url()
    else:
        with open('log.txt', 'r') as file:
            for line in file:
                print(line)
    
p_app = Process(target=run_app)
p_url = Process(target=print_url)
p_app.start()
p_url.start()
p_app.join()
p_url.join()